In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pkl

In [2]:
def sort_periodogram(per):
    local_maxima = np.where((per[1:-1] > per[:-2]) & (per[1:-1] > per[2:]))[0] + 1
    idx_highest = local_maxima[np.argsort(per[local_maxima])[::-1]]
    return idx_highest

In [3]:
def top_frequencies(top, idx_highest, freq):
    frequencies = [freq[i] for i in idx_highest]
    return frequencies[:top]

In [4]:
# tamaño max-1 + max-2
def generator_multiples(max):
    mult = np.concatenate([np.arange(1, max, 1), # multiples
                                1./np.arange(2, max, 1)]) # submultiples
    return mult

In [5]:
def relative_error(observed, actual):
    return np.absolute((observed - actual)/actual)

In [6]:
def matrix_band(frequencies_ranking, max, top, period, tol,multiples):
    matrix = [ [ [] for i in range(2*max-3) ] for j in range(top) ]
    for i in range(len(frequencies_ranking)):
        for j in range(len(multiples)):
            error = relative_error(1/(frequencies_ranking[i]*multiples[j]), period)
            tol_bool = error<tol
            matrix[i][j] = [error, tol_bool]
    return matrix

In [7]:
directory = '../dataset'
max_mult = 5
top = 10
tol = 1e-2
multiples = generator_multiples(max_mult)
data_global = pd.read_pickle('../dataset/data_global.pkl')
for folder_type in os.listdir(directory):
    # d_folder_type = 
    d_folder_type = os.path.join(directory, folder_type)
    # folder_type = 'ECL' 'RR'
    if folder_type.find('.') == -1:
        freq = pd.read_pickle('../dataset/'+folder_type+'/frequencies_'+folder_type+'.pkl')
        #lc_folder = '530580438478016768'
        for lc_folder in os.listdir(d_folder_type):
            # porque esta frequencies_ECL.pkl
            if lc_folder.find('.') == -1:
                # d_lc_folder
                dict_matrix = {}
                dict_matrix['multiples'] = multiples
                period = data_global[data_global.index == int(lc_folder)].values[0][0]
                d_lc_folder = os.path.join(d_folder_type,lc_folder)
                d_periodograms = os.path.join(d_lc_folder, 'periodograms.pkl')
                dict_periodograms = pd.read_pickle(d_periodograms)
                for band in dict_periodograms.keys():
                    per = dict_periodograms[band]
                    idx_highest = sort_periodogram(per)
                    freq_ranking = top_frequencies(top, idx_highest,freq)
                    matrix = matrix_band(freq_ranking,max_mult,top,period,tol, multiples)
                    dict_matrix[band] = [freq_ranking, matrix]
                with open(d_lc_folder+'/error_matrixes'+lc_folder+'.pkl', 'wb') as handle:
                    pkl.dump(dict_matrix, handle, protocol=pkl.HIGHEST_PROTOCOL)

        


In [8]:
def print_matrix(dict_matrix):
    print('Multiples: ', dict_matrix['multiples'])
    

In [9]:
def print_matrix(matrix, top, max, freq_ranking):
    for i in range(top):
        print(f'freq ranking {i+1}: {freq_ranking[i]:0.4f}', end='  ')
        for j in range(2*max-3):
            
            print(f"[{matrix[i][j][0]:0.4f}, {matrix[i][j][1]}]", end=" ")
        print()

In [11]:
name = '193720243577690880'
dict_matrix = pd.read_pickle('../dataset/ECL/'+name+'/error_matrixes'+name+'.pkl')
print('Multiples: ', dict_matrix['multiples'])
for band in ['BP', 'G']:
    if band != 'multiples':
        print('Band: '+band)
        matrix = dict_matrix[band][1]
        freq_ranking = dict_matrix[band][0]
        print_matrix(matrix,top,max_mult,freq_ranking)


Multiples:  [1.         2.         3.         4.         0.5        0.33333333
 0.25      ]
Band: BP
freq ranking 1: 2.0264  [0.8520, False] [0.9260, False] [0.9507, False] [0.9630, False] [0.7041, False] [0.5561, False] [0.4081, False] 
freq ranking 2: 1.6378  [0.8169, False] [0.9085, False] [0.9390, False] [0.9542, False] [0.6338, False] [0.4507, False] [0.2677, False] 
freq ranking 3: 0.0787  [2.8113, False] [0.9057, False] [0.2704, False] [0.0472, False] [6.6227, False] [10.4340, False] [14.2453, False] 
freq ranking 4: 1.8635  [0.8391, False] [0.9195, False] [0.9464, False] [0.9598, False] [0.6782, False] [0.5173, False] [0.3564, False] 
freq ranking 5: 0.4581  [0.3454, False] [0.6727, False] [0.7818, False] [0.8364, False] [0.3092, False] [0.9638, False] [1.6184, False] 
freq ranking 6: 1.7990  [0.8333, False] [0.9167, False] [0.9444, False] [0.9583, False] [0.6666, False] [0.5000, False] [0.3333, False] 
freq ranking 7: 0.3163  [0.0519, False] [0.5260, False] [0.6840, False] [0.

In [12]:
for band in ['RP', 'multiband']:
    if band != 'multiples':
        print('Band: '+band)
        matrix = dict_matrix[band][1]
        freq_ranking = dict_matrix[band][0]
        print_matrix(matrix,top,max_mult,freq_ranking)

Band: RP
freq ranking 1: 2.0264  [0.8520, False] [0.9260, False] [0.9507, False] [0.9630, False] [0.7041, False] [0.5561, False] [0.4081, False] 
freq ranking 2: 1.6378  [0.8169, False] [0.9085, False] [0.9390, False] [0.9542, False] [0.6338, False] [0.4507, False] [0.2677, False] 
freq ranking 3: 0.4581  [0.3454, False] [0.6727, False] [0.7818, False] [0.8364, False] [0.3092, False] [0.9638, False] [1.6184, False] 
freq ranking 4: 0.0787  [2.8113, False] [0.9057, False] [0.2704, False] [0.0472, False] [6.6227, False] [10.4340, False] [14.2453, False] 
freq ranking 5: 1.1512  [0.7395, False] [0.8698, False] [0.9132, False] [0.9349, False] [0.4791, False] [0.2186, False] [0.0419, False] 
freq ranking 6: 1.2686  [0.7636, False] [0.8818, False] [0.9212, False] [0.9409, False] [0.5273, False] [0.2909, False] [0.0545, False] 
freq ranking 7: 1.3332  [0.7751, False] [0.8875, False] [0.9250, False] [0.9438, False] [0.5502, False] [0.3253, False] [0.1003, False] 
freq ranking 8: 0.3163  [0.051

In [13]:
dict_matrix = {'multiples': [],
    'BP': [freq_ranking, matrix],
    'G': [freq_ranking, matrix],
    'RG': [freq_ranking, matrix],
    'multiband': [freq_ranking, matrix]
}